# 1 Concept of static representations

Short Intro and maybe a nice header image/teaser?

## Imports

In [ ]:
from pathlib import Path
from IPython.display import Code
from scripts.nb_utils import read_pc, read_from_output_folder, display_xml
import pyhelios
from pyhelios.util.xmldisplayer import find_playback_dir
import numpy as np
import pyvista as pv
pv.set_jupyter_backend('trame')
import laspy

## Example 1.1: Bi-temporal UAV-borne laser scanning of a growing forest

We start with a small homogeneous plot of trees that have been created with the Sapling Tree Gen add-on in Blender, which is based on the algorithm by Weber & Penn (1995). To generate the second time step, the trees are scaled in place in Blender.

#### The scene

This is how the XML scene file looks like:

In [3]:
Code(display_xml('data/scenes/forest_t1.xml', line_limit=34), language='XML')

<document>
    <scene id="forest_scene" name="Forest scene">
        <part>
            <filter type="objloader">
                <param type="string" key="filepath" value="data/sceneparts/forest_t1/Circle.obj" />
                <param type="string" key="up" value="z" />
            </filter>
            
            
        </part>
        <part>
            <filter type="objloader">
                <param type="string" key="filepath" value="data/sceneparts/forest_t1/tree1.obj" />
                <param type="string" key="up" value="z" />
            </filter>
            
            
        </part>
        <part>
            <filter type="objloader">
                <param type="string" key="filepath" value="data/sceneparts/forest_t1/tree2.obj" />
                <param type="string" key="up" value="z" />
            </filter>
            
            
        </part>
        <part>
            <filter type="objloader">
                <param type="string" key="filepath" value="data/sceneparts/forest_t1/tree3.obj" />
                <param type="string" key="up" value="z" />
            </filter>
            
            
        </part>

We can display the scene with pyvista: 

In [4]:
filenames1 = Path('data/sceneparts/forest_t1').glob('*.obj')
filenames2 = Path('data/sceneparts/forest_t2').glob('*.obj')

p = pv.Plotter(notebook=True, shape=(1, 2))
for filename in filenames1:
    mesh = pv.read(str(filename))
    p.add_mesh(mesh)
p.add_text('Forest mesh at t=1', color='k')
p.subplot(0, 1)
for filename in filenames2:
    mesh = pv.read(str(filename))
    p.add_mesh(mesh)
p.add_text('Forest mesh at t=2', color='k')
p.link_views()
p.camera_position = 'xz'
p.camera.zoom(1.8)
p.show()

Widget(value='<iframe src="http://localhost:58231/index.html?ui=P_0x27e4fbcd160_0&reconnect=auto" class="pyvis…

### The survey

The survey is a simple UAV-borne laser scanning (ULS) survey with three flight lines at an altitude of 75 m above ground level.

In [5]:
Code(display_xml('data/surveys/uls_forest_t1.xml'), language='XML')

<document>
	<scannerSettings id="uls_templ" active="true" pulseFreq_hz="100000" scanAngle_deg="90" scanFreq_hz="75" headRotatePerSec_deg="0.00" trajectoryTimeInterval_s="0.01" />
    <survey name="forest_t1" seed="42" scene="data/scenes/forest_t1.xml#forest_scene" platform="data/platforms.xml#copter_linearpath" scanner="data/scanners_als.xml#riegl_vux-1uav">
	<FWFSettings beamSampleQuality="3" binSize_ns="0.25" maxFullwaveRange_ns="100" winSize_ns="1.5" />
		<!-- platform: copter_linearpath, deflector: rotating -->
		
		<leg>
			<platformSettings x="-32" y="-20" z="75" onGround="false" movePerSec_m="5" />
			<scannerSettings template="uls_templ" />
		</leg>
		
		<leg>
			<platformSettings x="24" y="-20" z="75" onGround="false" movePerSec_m="5" />
			<scannerSettings template="uls_templ" active="false" />
		</leg>
        
        <leg>
			<platformSettings x="24" y="-6" z="75" onGround="false" movePerSec_m="5" />
			<scannerSettings template="uls_templ" />
		</leg>
		
		<leg>
			<platformSettings x="-32" y="-6" z="75" onGround="false" movePerSec_m="5" />
			<scannerSettings template="uls_templ" active="false" />
		</leg>
        
        		<leg>
			<platformSettings x="-32" y="8" z="75" onGround="false" movePerSec_m="5" />
			<scannerSettings template="uls_templ" />
		</leg>
		
		<leg>
			<platformSettings x="24" y="8" z="75" onGround="false" movePerSec_m="5" />
			<scannerSettings template="uls_templ" active="false" />
		</leg>

    </survey>
</document>

### Executing the simulations

In JupyterLab, we can run external commands with the !command syntax. We will use this to run the simulations for time steps 1 and 2, which are exactly the same except for the different scene.

In [7]:
!helios data/surveys/uls_forest_t1.xml -q --lasOutput --zipOutput --rebuildScene

The following code loads the points from the output folder and merges the flight lines.

In [8]:
survey_file = 'data/surveys/uls_forest_t1.xml'
output_path = find_playback_dir(survey_file)

print("Loading points from", Path(output_path))
pc_t1, object_id_t1, _, _, _, _ = read_from_output_folder(output_path)

Loading points from D:\VirtuaLearn3D\Publications\VLS-4D_vegetation\helios\output\forest_t1\2025-03-04_11-45-59


In [9]:
!helios data/surveys/uls_forest_t2.xml -q --lasOutput --zipOutput --rebuildScene

In [10]:
survey_file = 'data/surveys/uls_forest_t2.xml'
output_path = find_playback_dir(survey_file)

print("Loading points from", Path(output_path))
pc_t2, object_id_t2, _, _, _, _ = read_from_output_folder(output_path)

Loading points from D:\VirtuaLearn3D\Publications\VLS-4D_vegetation\helios\output\forest_t2\2025-03-04_11-47-45


### Visualizing the output

In [12]:
pl = pv.Plotter(shape=(1, 2))
# t1
actor = pl.add_points(pc_t1, scalars=object_id_t1, style='points', render_points_as_spheres=True, point_size=5)
pl.remove_scalar_bar()
pl.add_text('Forest at t=1', color='k')

# t2
pl.subplot(0, 1)
actor = pl.add_points(pc_t2, scalars=object_id_t2, style='points', render_points_as_spheres=True, point_size=5)
pl.remove_scalar_bar()
pl.add_text('Forest at t=2', color='k')

pl.link_views()
pl.camera_position = 'xz'
pl.camera.zoom(1.8)
pl.show()

Widget(value='<iframe src="http://localhost:58231/index.html?ui=P_0x27e0293f610_2&reconnect=auto" class="pyvis…

## Example 1.2: Terrestrial laser scanning (TLS) time series of tree defoliation (swap-on-repeat)

The concept of static representations can also be implemented in an easier way using *swap-on-repeat*. This [HELIOS++ feature](https://github.com/3dgeo-heidelberg/helios/wiki/Scene-swaps) allows to define a dynamic scene of several snapshots in a single scene XML. In this XML, a scene part can have several "swaps" for a defined number of epochs, where the scene part geometry can be exchanged, or transformation filters can be applied.
We will use this to create a small time series of a tree which undergoes defoliation.

### The scene

This is what the scene looks like:

In [11]:
Code(display_xml('data/scenes/tree_defoliation.xml'), language='XML')

<document>
    <scene id="swap_scene" name="Dynamic scene">
        
        <part>
            <filter type="objloader">
                <param type="string" key="filepath" value="data/sceneparts/tree_defoliation/wood.obj" />
                <param type="string" key="up" value="z" />
            </filter>
            <!-- two more steps with the same wood" --> 
            <swap swapStep="2">
            <filter type="objloader">
                <param type="string" key="filepath" value="data/sceneparts/tree_defoliation/wood.obj" />
                <param type="string" key="up" value="z" />
            </filter>
            </swap>
            <!-- swap fine wood structure by just the main banches (-> branch dieback) for the last step -->
            <swap swapStep="1">
            <filter type="objloader">
                <param type="string" key="filepath" value="data/sceneparts/tree_defoliation/wood_reduced.obj" />
                <param type="string" key="up" value="z" />
            </filter>
            </swap>
        </part>
        <part>
            <filter type="objloader">
                <param type="string" key="filepath" value="data/sceneparts/tree_defoliation/leaves_green.obj" />
                <param type="string" key="up" value="z" />
            </filter>
            <!-- swap green leaves for yellow leaves -->
            <swap swapStep="1">
            <filter type="objloader">
                <param type="string" key="filepath" value="data/sceneparts/tree_defoliation/leaves_yellow.obj" />
                <param type="string" key="up" value="z" />
            </filter>
            </swap>
            <!-- no leaves at all for two steps -->
            <swap swapStep="2" force_null="true">
            </swap>
        </part>
    </scene>
</document>

The tree has been reconstructed from a real point cloud using [TreeQSM](https://github.com/InverseTampere/TreeQSM) and the leaves have been added using [QSM-FanNNI](https://github.com/InverseTampere/qsm-fanni-matlab). Let's have a quick look at the scene parts in 3D.

In [19]:
p = pv.Plotter(notebook=True, shape=(2, 2), window_size=(600, 600))
mesh_wood = pv.read('data/sceneparts/tree_defoliation/wood.obj')
p.add_mesh(mesh_wood)
p.add_text('Wood', color='k', font_size=12)
p.subplot(0, 1)
mesh = pv.read('data/sceneparts/tree_defoliation/leaves_green.obj')
p.add_mesh(mesh)
p.add_text('Leaves green', color='k', font_size=12)
p.subplot(1, 0)
mesh = pv.read('data/sceneparts/tree_defoliation/wood_reduced.obj')
p.add_mesh(mesh)
p.add_text('Wood reduced', color='k', font_size=12)
p.subplot(1, 1)
mesh = pv.read('data/sceneparts/tree_defoliation/leaves_yellow.obj')
p.add_mesh(mesh)
p.add_text('Leaves yellow and smaller', color='k', font_size=12)
p.link_views()
p.camera_position = 'xz'
p.camera.zoom(1)
p.show()

Widget(value='<iframe src="http://localhost:58231/index.html?ui=P_0x27e18485810_9&reconnect=auto" class="pyvis…

### The survey

The survey is a simple multi-station terrestrial laser scanning (TLS) setup with five scan positions:

In [20]:
Code(display_xml('data/surveys/defoliation_time_series.xml'), language='XML')

<document>
        <scannerSettings id="tls" active="true" pulseFreq_hz="300000" verticalResolution_deg="0.034" horizontalResolution_deg="0.034" />
        <survey name="tree_defoliation" platform="data/platforms.xml#tripod" scanner="data/scanners_tls.xml#riegl_vz400" scene="data/scenes/tree_defoliation.xml#swap_scene">
        <leg>
            <platformSettings x="-15" y="0" z="0" />
            <scannerSettings template="tls" headRotateStart_deg="240" headRotateStop_deg="300" trajectoryTimeInterval_s="0.05" />
        </leg>
        <leg>
            <platformSettings x="-4.4" y="11.7" z="0" />
            <scannerSettings template="tls" headRotateStart_deg="195" headRotateStop_deg="255" trajectoryTimeInterval_s="0.05" />
        </leg>
        <leg>
            <platformSettings x="14.3" y="6" z="0" />
            <scannerSettings template="tls" headRotateStart_deg="105" headRotateStop_deg="165" trajectoryTimeInterval_s="0.05" />
        </leg>
        <leg>
            <platformSettings x="12.3" y="-11.8" z="0" />
            <scannerSettings template="tls" headRotateStart_deg="15" headRotateStop_deg="75" trajectoryTimeInterval_s="0.05" />
        </leg>
        <leg>
            <platformSettings x="-9.4" y="-12.6" z="0" />
            <scannerSettings template="tls" headRotateStart_deg="285" headRotateStop_deg="345" trajectoryTimeInterval_s="0.05" />
        </leg>
        
        </survey>
    </document>

### Executing the simulations

This time, we only have to execute the survey once, and it will automatically be repeated for as many swaps as have been detected in the scene XML file (in this case: 4).

In [ ]:
!helios data/surveys/defoliation_time_series.xml -q --lasOutput --zipOutput --rebuildScene

In [21]:
output_path = Path(find_playback_dir('data/surveys/defoliation_time_series.xml')).parent
outfolders = list(Path(output_path).glob('2025*'))[-4:]

pc_t1, _, classification_t1, amplitude_t1, _, _ = read_from_output_folder(outfolders[0])
pc_t2, _, classification_t2, amplitude_t2, _, _ = read_from_output_folder(outfolders[1])
pc_t3, _, classification_t3, amplitude_t3, _, _ = read_from_output_folder(outfolders[2])
pc_t4, _, classification_t4, amplitude_t4, _, _ = read_from_output_folder(outfolders[3])

### Visualizing the Output

Let's have a look at the four output point clouds that we have simulated:

In [22]:
# TODO: better color scale, or filter by amplitude values beforehand?
p = pv.Plotter(notebook=True, shape=(2, 2), window_size=(600, 600))
p.add_points(pc_t1, scalars=amplitude_t1, style='points', cmap='gwv', point_size=1, scalar_bar_args={'title': 'amplitude | 1'}, show_scalar_bar=False)
p.subplot(0, 1)
p.add_points(pc_t2, scalars=amplitude_t2, style='points', cmap='gwv', point_size=1, scalar_bar_args={'title': 'amplitude | 2'}, show_scalar_bar=False)
p.subplot(1, 0)
p.add_points(pc_t3, scalars=amplitude_t3, style='points', cmap='gwv', point_size=1, scalar_bar_args={'title': 'amplitude | 3'}, show_scalar_bar=False)
p.subplot(1, 1)
p.add_points(pc_t4, scalars=amplitude_t4, style='points', cmap='gwv', point_size=1, scalar_bar_args={'title': 'amplitude | 4'}, show_scalar_bar=False)
#p.add_scalar_bar(
#    'helios amplitude',
#    vertical=False,
#    title_font_size=20,
#    label_font_size=16,
#    outline=False,
#    fmt='%10.1f',
#    n_labels=0
#)
p.link_views()
p.camera_position = 'xz'
p.camera.zoom(1)
p.show()

Widget(value='<iframe src="http://localhost:58231/index.html?ui=P_0x27e0293cf50_10&reconnect=auto" class="pyvi…